<a href="https://colab.research.google.com/github/edoardorossi1995/Tesi/blob/main/Average_Reward_Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pseudocodice 
###Algoritmo di tipo Q-learning, off policy, a task non episodico, che approssima la funzione valore con una rete neurale, con un framework di tipo average reward.

Inizializza la rete neurale per approssimare la funzione Q

Inizializza la media delle ricompense future attese per ogni stato-azione a zero

Ripeti finché non raggiungi la fine del processo:

- S = stato corrente

- Seleziona un'azione A seguendo la politica di esplorazione (ad esempio, la politica ε-greedy)
- Esegui l'azione A e ottieni il prossimo stato S' e la ricompensa R
- Calcola la media delle ricompense future attese per lo stato-azione corrente utilizzando la formula:
- - media_ricompense[S, A] = (1 - α) * media_ricompense[S, A] + α * (R + γ * max(Q(S', a), rete_neurale))
- Calcola il valore target utilizzando la formula:
- - target = media_ricompense[S, A]
- Aggiorna la rete neurale utilizzando il valore target come obiettivo e il vettore di stato-azione come input S = S'

Restituisci la **rete neurale** ottenuta







#Drive Setting


In [1]:
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

if IN_COLAB == True:
  from google.colab import drive
  drive.mount('/content/gdrive')
  %cd /content/gdrive/MyDrive/Tesi
  #!ls

import warnings
import pickle
import sys
sys.path.insert(0, '/content/gdrive/MyDrive/Tesi/functions')
sys.path.insert(0, '/content/gdrive/MyDrive/Tesi')

warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from keras.models import Sequential
from keras.layers import Dense, Lambda
from keras.activations import exponential, relu
from keras import backend as K

from pkl import store_data, load_data
from functions.compressor import compress

import random
from collections import defaultdict

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/MyDrive/Tesi


# Import Anagrafica e CTA

In [2]:
#import pandas as pd
#import numpy as np
#import matplotlib.pyplot as plt

CTA30_def = ['/content/gdrive/MyDrive/Tesi/CTA30/DF_clean/clean_CTA_30_def.csv']
df_def = pd.read_csv(CTA30_def[0])

CTA30_18 = ['/content/gdrive/MyDrive/Tesi/CTA30/DF_clean/clean_CTA_30_2018.csv']
df = pd.read_csv(CTA30_18[0])

pca_df = compress(df)

In [3]:
print("Anagrafica shape:",df_def.shape,"\nDataframe shape:", df.shape,"\nPca Dataframe shape:", pca_df.shape)

Anagrafica shape: (76, 2) 
Dataframe shape: (141811, 76) 
Pca Dataframe shape: (141811, 7)


In [4]:
df_def

,pointColName,pointName
0,AUTOMAN_SERR_CMD_00,Automatico Manuale Serrande Comando
1,AUTOMAN_VLV_CMD_00,Automatico Manuale Valvole Comando
2,AUTOMAN_M_CMD_00,Automatico Manuale Mandata Comando
3,AUTOMAN_R_CMD_00,Automatico Manuale Ripresa Comando
4,LOCREM_CMD_00,Selettore Locale Remoto Comando
...,...,...
71,VOLT_R_00,Tensione Ripresa
72,T_M_SP_00,Temperatura Aria Mandata Setpoint
73,T_R_SP_00,Temperatura Aria Ritorno Setpoint
74,T_AMB_04,Temperatura Aria Ambiente Media


In [5]:
df.loc[11700:11900,'VLV_RAF_FDBK_00']

11700    0.0
11701    0.0
11702    0.0
11703    0.0
11704    0.0
        ... 
11896    0.0
11897    0.0
11898    0.0
11899    0.0
11900    0.0
Name: VLV_RAF_FDBK_00, Length: 201, dtype: float64

# Rete Neurale

Log-sum-exp neural networks and posynomial
models for convex and log-log-convex data 

(Calafiore, Gaubert, Possieri)

In [6]:
n = load_data('*.pkl')
action_vector_dimension = 2

# Funzione di attivazione logatirmica (ultimo strato)
def log_act(x):
  return K.log(x)

# Architettura rete neurale
input_dim = n['n_components']+action_vector_dimension
inner_dim_1 = 32
output_dim = 1

print("input dim: ", input_dim)
print("inner layer dim: ", inner_dim_1)
print("output dim: ", output_dim)

# Costruzione strati
rete_neurale = Sequential()
rete_neurale.add(Dense(units = inner_dim_1, activation = exponential, input_dim = input_dim))
rete_neurale.add(Dense(units = output_dim, activation=log_act)) 

rete_neurale.compile(optimizer = 'adam', loss = 'mean_squared_error', metrics = ['accuracy'])

input dim:  9
inner layer dim:  32
output dim:  1


# Q-Learning Agent

In [7]:
# Dimensione problema e inizializzazione stato iniziale
[S, feat] = df.shape
current_state = 1500#np.random.randint(0,S-1)

# Inizializza la media delle ricompense future attese per ogni stato-azione a zero
q = defaultdict(float)

# Stato corrente
s = current_state

# Parametri di apprendimento (learning rate, discount factor)
alpha = 0.5
gamma = 1

# Media mobile esponenziale (avr = avr + alpha * (r - avr)) 
# [scelta perché computazionalmente più leggera di una media aritmetica]
average_reward = 0

# Setting Episodi
total_episodes = 3
current_episode = 0

# Ripeti finché non raggiungi la fine del processo
for current_episode in range(total_episodes):
    
    # Stato successivo e ottieni l'azione A e la ricompensa R
    s_prime = s+1
    if s_prime > S-1:
      print("s =",s)
      break

    #a1 = df.loc[s,'F_M_FDBK_00']-df.loc[s_prime,'F_M_FDBK_00']
    a1 = df.loc[s_prime,'F_M_FDBK_00']
    #a2 = df.loc[s,'VLV_RAF_FDBK_00']-df.loc[s_prime,'VLV_RAF_FDBK_00']
    a2 = df.loc[s_prime,'VLV_RAF_FDBK_00']
    a = (a1,a2)
    r = -df.loc[s+1,'PW_ATT_M_00']

    average_reward = average_reward + alpha * (r - average_reward)

    # Comprimo s_prime per generare un valore di input_prime per la predizione di q(s',a')
    pca_s_prime = pca_df.loc[s_prime,:]
    a1_prime = df.loc[s_prime+1,'F_M_FDBK_00']
    a2_prime = df.loc[s_prime+1,'VLV_RAF_FDBK_00']
    a_prime = (a1_prime, a2_prime)
    
    nn_input_prime = np.concatenate((pca_s_prime.values, np.array([a_prime[0],a_prime[1]])))
    nn_input_prime = np.reshape(nn_input_prime, (1,input_dim))

    # Calcola la media delle ricompense future attese per lo stato-azione corrente
    # q = q + a*(avr + gamma*max(q') - q)
    print("pre q(s,a) = ",q[(s,a)])
    print("nn_input_prime = ",nn_input_prime)
    if current_episode != 0:
      #print("nn = ",rete_neurale.predict(nn_input_prime))
      q[(s,a)] = q[(s,a)] + alpha * (average_reward + gamma * rete_neurale.predict(nn_input_prime) - q[(s,a)])
    print("q(s,a) = ",q[(s,a)])

    # Calcola il valore target
    target = q[(s,a)]
    
    # Comprimo lo stato e genero l'input della rete neurale
    pca_s = pca_df.loc[s,:]

    nn_input = np.concatenate((pca_s.values, np.array([a[0],a[1]])))
    nn_input = np.reshape(nn_input, (1,input_dim))
    target = np.array([target])

    # Aggiorna la rete neurale
    rete_neurale.fit(nn_input, target, epochs=1, verbose=0)

    # Aggiorna lo stato corrente
    s = s_prime

    # Plot statistiche ogni 100 episodi
    if current_episode%1 == 0:
      print("Episodio ",current_episode,"/",total_episodes)
      print("target: ", target, "\nreward: ",r,"\naverage_reward: ",np.mean(average_reward))
      print("\n")

# Restituisci la rete neurale ottenuta
#return rete_neurale

pre q(s,a) =  0.0
nn_input_prime =  [[ 5.71791967e-02  1.94945619e+00 -5.88307401e-01  1.39295974e+00
   5.25227032e-01  3.58965728e-01  4.79149861e-02  1.00043600e+02
   0.00000000e+00]]
q(s,a) =  0.0
Episodio  0 / 3
target:  [0.] 
reward:  -19.5972 
average_reward:  -9.7986


pre q(s,a) =  0.0
nn_input_prime =  [[ 5.77441305e-02  1.95148783e+00 -5.88245303e-01  1.39377315e+00
   5.26320708e-01  3.59644591e-01  4.62829345e-02  9.99862000e+01
   0.00000000e+00]]
1/1 [==============================] - 0s 156ms/step
nn =  [[nan]]
1/1 [==============================] - 0s 52ms/step
q(s,a) =  [[nan]]
Episodio  1 / 3
target:  [[[nan]]] 
reward:  -19.7198 
average_reward:  -14.7592


pre q(s,a) =  0.0
nn_input_prime =  [[ 5.66001871e-02  1.94848598e+00 -5.88357199e-01  1.39294421e+00
   5.25273169e-01  3.57556411e-01  4.65486792e-02  1.00002200e+02
   0.00000000e+00]]
1/1 [==============================] - 0s 34ms/step
nn =  [[nan]]
1/1 [==============================] - 0s 66ms/step
q(s,a) 

In [8]:
    nn_input = np.concatenate((pca_s.values, np.array([a[0],a[1]])))
    nn_input = np.reshape(nn_input, (1,input_dim))
    target = np.array([target])
    target

array([[[[nan]]]], dtype=float32)

In [9]:
q

defaultdict(float,
            {(1500, (100.0074, 0.0)): 0.0,
             (1501, (100.0436, 0.0)): array([[nan]], dtype=float32),
             (1502, (99.9862, 0.0)): array([[nan]], dtype=float32)})